# CNN

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Input, BatchNormalization, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

np.random.seed(42)
tf.random.set_seed(42)

## Define the evaluation functions

In [2]:
def calculate_mape(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def calculate_mmre(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    return np.mean(np.abs(y_true[mask] - y_pred[mask]) / y_true[mask])

def calculate_mdmre(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    return np.median(np.abs(y_true[mask] - y_pred[mask]) / y_true[mask])

def calculate_pred25(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    mre = np.abs(y_true[mask] - y_pred[mask]) / y_true[mask]
    return np.mean(mre <= 0.25) * 100

## Read file and declare columns

In [3]:
df = pd.read_csv('desharnais1.1_processed_corrected.csv')


numeric_columns = ['TeamExp', 'ManagerExp', 'YearEnd', 'Length', 'Effort', 
                   'Transactions', 'Entities', 'Adjustment', 'PointsAjust',
                   'StartYear', 'ProjectDurationYears', 'Transactions_Entities',
                   'Effort_PointsAjust', 'Effort_per_PointsAjust', 'Transactions_per_Entities']
binary_columns = ['Language_b\'1\'', 'Language_b\'2\'', 'Language_b\'3\'', 'HighEffort', 'HighPointsAjust']



## Create feature set and target variable.

In [4]:
features = numeric_columns + binary_columns
X = df[features].values
y = df['Effort'].values

## Data enhancement with Gaussian noise

Data augmentation by adding light Gaussian noise to the input data X aims to: 

- Improve the model's generalization, 

- Combat overfitting (memorizing the training data), 

- Increase data diversity without needing to collect more.

In [5]:
def add_gaussian_noise(X, noise_factor=0.01):
    if not np.issubdtype(X.dtype, np.number):
        raise ValueError("Input X must be numeric.")
    if np.any(np.isnan(X)):
        raise ValueError("Input X contains NaN values.")
    noise = np.random.normal(loc=0, scale=noise_factor, size=X.shape)
    return X + noise

X_augmented = X.copy()
y_augmented = y.copy()
for _ in range(2):  
    X_noisy = add_gaussian_noise(X, noise_factor=0.01)
    X_augmented = np.vstack((X_augmented, X_noisy))
    y_augmented = np.hstack((y_augmented, y))

print("\n=== After data augmentation with Gaussian noise ===")
print("X_augmented shape:", X_augmented.shape)
print("y_augmented shape:", y_augmented.shape)


=== After data augmentation with Gaussian noise ===
X_augmented shape: (243, 20)
y_augmented shape: (243,)


## Reshape and Split data

In [6]:
X_augmented = X_augmented.reshape(X_augmented.shape[0], X_augmented.shape[1], 1)

print("\n=== Data size after reshape ===")
print("X_augmented shape:", X_augmented.shape)
print("y_augmented shape:", y_augmented.shape)

X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_augmented, test_size=0.15, random_state=42)

print(f"\n✅ Data size of CNN:")
print(f" - X_train: {X_train.shape}")
print(f" - X_test : {X_test.shape}")


=== Data size after reshape ===
X_augmented shape: (243, 20, 1)
y_augmented shape: (243,)

✅ Data size of CNN:
 - X_train: (206, 20, 1)
 - X_test : (37, 20, 1)


- CNN requires at least 3 dimensions of input:
With tabular or vector data, you need to add a channel dimension for CNN to understand that it is a spatial dimension or image channel

- Use an 85% training and 15% testing ratio with random_state=42 for replication.

## Building the CNN model

Define the function to build a CNN model with Conv1D, BatchNormalization, Flatten, Dense, and Dropout layers.

- The model uses Conv1D layers with the ReLU activation function, batch normalization, and Dropout to avoid overfitting. 
- It uses the Huber loss function and is optimized with Adam.

In [7]:
def build_cnn_model(filters=8, l2_reg=0.01, dense_units=16, dropout_rate=0.3, learning_rate=0.001):
    l2_reg = max(l2_reg, 0.001)
    model = Sequential([
        Input(shape=(X_train.shape[1], X_train.shape[2])),
        Conv1D(filters, kernel_size=2, activation='relu', padding='same', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Conv1D(filters, kernel_size=2, activation='relu', padding='same', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Flatten(),
        Dense(dense_units, activation='relu', kernel_regularizer=l2(l2_reg)),
        Dropout(dropout_rate),
        Dense(1, activation='linear')
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])
    return model


## Definition of hyperparameter space and PSO function

Define the hyperparameter space and the supporting functions for the Particle Swarm Optimization (PSO) algorithm.

- Define the limits for hyperparameters (filters, l2_reg, dense_units, dropout_rate, learning_rate, batch_size, epochs). 
- The **random_particle** function creates a random particle. 
- The **decode_particle** function converts the particle into model parameters. 
- The **fitness_function** evaluates the model's performance using the average RMSE through K-Fold.

In [8]:
param_bounds = {
    'filters': (4, 16),
    'l2_reg': (0.001, 0.05),
    'dense_units': (8, 32),
    'dropout_rate': (0.2, 0.4),
    'learning_rate': (1e-4, 1e-2),
    'batch_size': (16, 32),
    'epochs': (50, 100)
}

# Hàm mã hóa & giải mã particle
def random_particle():
    return np.array([
        np.random.randint(param_bounds['filters'][0], param_bounds['filters'][1] + 1),
        np.random.uniform(param_bounds['l2_reg'][0], param_bounds['l2_reg'][1]),
        np.random.randint(param_bounds['dense_units'][0], param_bounds['dense_units'][1] + 1),
        np.random.uniform(param_bounds['dropout_rate'][0], param_bounds['dropout_rate'][1]),
        np.random.uniform(param_bounds['learning_rate'][0], param_bounds['learning_rate'][1]),
        np.random.randint(param_bounds['batch_size'][0], param_bounds['batch_size'][1] + 1),
        np.random.randint(param_bounds['epochs'][0], param_bounds['epochs'][1] + 1)
    ])

def decode_particle(particle):
    params = {
        'filters': int(particle[0]),
        'l2_reg': particle[1],
        'dense_units': int(particle[2]),
        'dropout_rate': particle[3],
        'learning_rate': particle[4],
        'batch_size': int(particle[5]),
        'epochs': int(particle[6])
    }
    params['l2_reg'] = max(params['l2_reg'], 0.001)
    params['l2_reg'] = min(params['l2_reg'], param_bounds['l2_reg'][1])
    return params

# Hàm fitness cho PSO
def fitness_function(particle):
    params = decode_particle(particle)
    model = build_cnn_model(**{k: v for k, v in params.items() if k != 'batch_size' and k != 'epochs'})
    
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    rmse_scores = []
    
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
        
        model.fit(X_tr, y_tr, epochs=params['epochs'], batch_size=params['batch_size'], 
                validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
        y_pred = model.predict(X_val, verbose=0)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_scores.append(rmse)
    
    return np.mean(rmse_scores)

## Implementing the PSO algorithm

Using the PSO algorithm to find the optimal hyperparameter set. 
- Initialize num_particles=15 and max_iter=10. 

- Update the position and velocity of the particles based on the PSO formula. 

- Find the best particle (g_best_position) and the best score (g_best_score).

In [9]:
def run_pso_cnn(num_particles=15, max_iter=10):
    dim = len(param_bounds)
    bounds_array = np.array(list(param_bounds.values()))
    
    particles = [random_particle() for _ in range(num_particles)]
    velocities = [np.zeros(dim) for _ in range(num_particles)]
    
    p_best_positions = particles.copy()
    p_best_scores = [fitness_function(p) for p in particles]
    
    g_best_index = np.argmin(p_best_scores)
    g_best_position = p_best_positions[g_best_index]
    g_best_score = p_best_scores[g_best_index]
    
    w, c1, c2 = 0.5, 1.5, 1.5
    
    for iter in range(max_iter):
        print(f"\n🔁 Iteration {iter + 1}/{max_iter}")
        for i in range(num_particles):
            r1 = np.random.rand(dim)
            r2 = np.random.rand(dim)
            
            velocities[i] = (
                w * velocities[i]
                + c1 * r1 * (p_best_positions[i] - particles[i])
                + c2 * r2 * (g_best_position - particles[i])
            )
            
            particles[i] += velocities[i]
            particles[i] = np.clip(particles[i], bounds_array[:, 0], bounds_array[:, 1])
            particles[i][1] = max(particles[i][1], param_bounds['l2_reg'][0])
            particles[i][1] = min(particles[i][1], param_bounds['l2_reg'][1])
            particles[i][3] = np.clip(particles[i][3], param_bounds['dropout_rate'][0], param_bounds['dropout_rate'][1])
            
            score = fitness_function(particles[i])
            
            if score < p_best_scores[i]:
                p_best_scores[i] = score
                p_best_positions[i] = particles[i]
                
            if score < g_best_score:
                g_best_score = score
                g_best_position = particles[i]
                print(f"✅ Cập nhật g_best: Score = {g_best_score:.4f}")
    
    return g_best_position, g_best_score

# Chạy PSO
print("🚀 Chạy PSO để tìm siêu tham số tối ưu...")
best_particle, best_score = run_pso_cnn(num_particles=15, max_iter=10)
best_params = decode_particle(best_particle)
print(f"🏆 Siêu tham số tốt nhất: {best_params}")
print(f"📉 Score tốt nhất: {best_score:.4f}")

🚀 Chạy PSO để tìm siêu tham số tối ưu...

🔁 Iteration 1/10
✅ Cập nhật g_best: Score = 0.1535
✅ Cập nhật g_best: Score = 0.1367

🔁 Iteration 2/10


KeyboardInterrupt: 

## Training the optimal model

Use the best hyperparameters to train the CNN model with K-Fold cross-validation. 
Details: 
- Use K-Fold with 3 folds. 
- Train the model with EarlyStopping and ReduceLROnPlateau to optimize. 
- Calculate the average RMSE across the folds.

In [ ]:
model_optimal = build_cnn_model(**{k: v for k, v in best_params.items() if k != 'batch_size' and k != 'epochs'})
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

kf = KFold(n_splits=3, shuffle=True, random_state=42)
rmse_scores_optimal = []
history_all = {'loss': [], 'val_loss': []}

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"\n📂 Fold {fold + 1}/3")
    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]
    
    history = model_optimal.fit(X_tr, y_tr, epochs=best_params['epochs'], batch_size=best_params['batch_size'], 
                            validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
    y_pred = model_optimal.predict(X_val, verbose=0)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores_optimal.append(rmse)
    print(f"✅ Fold {fold + 1} RMSE: {rmse:.4f}")
    
    # Lưu lịch sử huấn luyện
    history_all['loss'].append(history.history['loss'])
    history_all['val_loss'].append(history.history['val_loss'])

print(f"\n📊 RMSE trung bình qua 3 folds: {np.mean(rmse_scores_optimal):.4f}")


📂 Fold 1/3
✅ Fold 1 RMSE: 0.1435

📂 Fold 2/3
✅ Fold 2 RMSE: 0.1316

📂 Fold 3/3
✅ Fold 3 RMSE: 0.0786

📊 RMSE trung bình qua 3 folds: 0.1179


## Evaluate the model on the test set.

Evaluate the model on the test set and calculate metrics such as MSE, RMSE, MAE, R², MAPE, MMRE, MdMRE, PRED(25).

In [ ]:
max_len = max(len(h) for h in history_all['loss'])
loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['loss']], axis=0)
val_loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['val_loss']], axis=0)

# Đánh giá trên tập test
y_pred = model_optimal.predict(X_test, verbose=0).flatten()

# Tính các chỉ số đánh giá
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = calculate_mape(y_test, y_pred)
mmre = calculate_mmre(y_test, y_pred)
mdmre = calculate_mdmre(y_test, y_pred)
pred25 = calculate_pred25(y_test, y_pred)

# Đánh giá bootstrap
n_bootstraps = 500
bootstrap_metrics = {'mse': [], 'mae': [], 'r2': [], 'mape': [], 'mmre': [], 'mdmre': [], 'pred25': []}

for _ in range(n_bootstraps):
    indices = np.random.choice(len(y_test), len(y_test), replace=True)
    y_test_boot = y_test[indices]
    y_pred_boot = y_pred[indices]
    bootstrap_metrics['mse'].append(mean_squared_error(y_test_boot, y_pred_boot))
    bootstrap_metrics['mae'].append(mean_absolute_error(y_test_boot, y_pred_boot))
    bootstrap_metrics['r2'].append(r2_score(y_test_boot, y_pred_boot))
    bootstrap_metrics['mape'].append(calculate_mape(y_test_boot, y_pred_boot))
    bootstrap_metrics['mmre'].append(calculate_mmre(y_test_boot, y_pred_boot))
    bootstrap_metrics['mdmre'].append(calculate_mdmre(y_test_boot, y_pred_boot))
    bootstrap_metrics['pred25'].append(calculate_pred25(y_test_boot, y_pred_boot))

# In kết quả
print("\n📈 Kết quả đánh giá bootstrap (trên giá trị đã scale):")
print(f"📌 MSE     : {np.mean(bootstrap_metrics['mse']):.4f} ± {np.std(bootstrap_metrics['mse']):.4f}")
print(f"📌 RMSE    : {np.mean(np.sqrt(bootstrap_metrics['mse'])):.4f} ± {np.std(np.sqrt(bootstrap_metrics['mse'])):.4f}")
print(f"📌 MAE     : {np.mean(bootstrap_metrics['mae']):.4f} ± {np.std(bootstrap_metrics['mae']):.4f}")
print(f"📌 R²      : {np.mean(bootstrap_metrics['r2']):.4f} ± {np.std(bootstrap_metrics['r2']):.4f}")
print(f"📌 MAPE    : {np.mean(bootstrap_metrics['mape']):.2f}% ± {np.std(bootstrap_metrics['mape']):.2f}%")
print(f"📌 MMRE    : {np.mean(bootstrap_metrics['mmre']):.4f} ± {np.std(bootstrap_metrics['mmre']):.4f}")
print(f"📌 MdMRE   : {np.mean(bootstrap_metrics['mdmre']):.4f} ± {np.std(bootstrap_metrics['mdmre']):.4f}")
print(f"📌 PRED(25): {np.mean(bootstrap_metrics['pred25']):.2f}% ± {np.std(bootstrap_metrics['pred25']):.2f}%")

# Lưu kết quả đánh giá
results = {
    'MSE': mse,
    'RMSE': rmse,
    'MAE': mae,
    'R2': r2,
    'MAPE': mape,
    'MMRE': mmre,
    'MdMRE': mdmre,
    'PRED(25)': pred25,
    'Bootstrap_MSE_Mean': np.mean(bootstrap_metrics['mse']),
    'Bootstrap_MSE_Std': np.std(bootstrap_metrics['mse']),
    'Bootstrap_MAE_Mean': np.mean(bootstrap_metrics['mae']),
    'Bootstrap_MAE_Std': np.std(bootstrap_metrics['mae']),
    'Bootstrap_R2_Mean': np.mean(bootstrap_metrics['r2']),
    'Bootstrap_R2_Std': np.std(bootstrap_metrics['r2']),
    'Bootstrap_MAPE_Mean': np.mean(bootstrap_metrics['mape']),
    'Bootstrap_MAPE_Std': np.std(bootstrap_metrics['mape']),
    'Bootstrap_MMRE_Mean': np.mean(bootstrap_metrics['mmre']),
    'Bootstrap_MMRE_Std': np.std(bootstrap_metrics['mmre']),
    'Bootstrap_MdMRE_Mean': np.mean(bootstrap_metrics['mdmre']),
    'Bootstrap_MdMRE_Std': np.std(bootstrap_metrics['mdmre']),
    'Bootstrap_PRED25_Mean': np.mean(bootstrap_metrics['pred25']),
    'Bootstrap_PRED25_Std': np.std(bootstrap_metrics['pred25'])
}

results_df = pd.DataFrame([results])
results_df.to_csv('cnn_evaluation_results_scaled.csv', index=False)
print("\nĐã lưu kết quả đánh giá vào 'cnn_evaluation_results_scaled.csv'")

# Trực quan hóa kết quả
plt.figure(figsize=(15, 12))

# Loss trung bình qua các folds
plt.subplot(2, 2, 1)
plt.plot(loss_avg, label='Training Loss')
plt.plot(val_loss_avg, label='Validation Loss')
plt.title('Average Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Huber Loss')
plt.legend()

# Predicted vs Actual
plt.subplot(2, 2, 2)
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.title('Predicted vs Actual Effort (Scaled)')
plt.xlabel('Actual Effort (Scaled)')
plt.ylabel('Predicted Effort (Scaled)')

# Error Distribution
errors = y_test - y_pred
plt.subplot(2, 2, 3)
sns.histplot(errors, kde=True)
plt.title('Error Distribution')
plt.xlabel('Prediction Error (Scaled)')
plt.ylabel('Frequency')

# Bootstrap RMSE
plt.subplot(2, 2, 4)
sns.boxplot(y=np.sqrt(bootstrap_metrics['mse']))
plt.title('Bootstrap RMSE Distribution (Scaled)')
plt.ylabel('RMSE (Scaled)')

plt.tight_layout()
plt.savefig('cnn_visualization_results_scaled.png')
plt.close()
print("\nĐã lưu hình ảnh trực quan hóa vào 'cnn_visualization_results_scaled.png'")


📈 Kết quả đánh giá bootstrap (trên giá trị đã scale):
📌 MSE     : 0.0527 ± 0.0262
📌 RMSE    : 0.2222 ± 0.0576
📌 MAE     : 0.1272 ± 0.0295
📌 R²      : 0.9623 ± 0.0169
📌 MAPE    : 56.05% ± 32.00%
📌 MMRE    : 0.5605 ± 0.3200
📌 MdMRE   : 0.1296 ± 0.0960
📌 PRED(25): 64.25% ± 11.67%

Đã lưu kết quả đánh giá vào 'cnn_evaluation_results_scaled.csv'

Đã lưu hình ảnh trực quan hóa vào 'cnn_visualization_results_scaled.png'


# LSTM

## Import libraries and set the seed.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Thiết lập seed để tái lập
np.random.seed(42)
tf.random.set_seed(42)

## Define the evaluation functions.

In [ ]:
def calculate_mape(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def calculate_mmre(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    return np.mean(np.abs(y_true[mask] - y_pred[mask]) / y_true[mask])

def calculate_mdmre(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    return np.median(np.abs(y_true[mask] - y_pred[mask]) / y_true[mask])

def calculate_pred25(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    mre = np.abs(y_true[mask] - y_pred[mask]) / y_true[mask]
    return np.mean(mre <= 0.25) * 100

## Define custom Transformers.

Define 4 custom Transformer layers for data preprocessing: 
- **DataTypeConverter**: Convert data types of numeric and binary columns. 
- **NaNImputer**: Fill missing values (NaN) with the mean of the numeric column. 
- **SelectiveScaler**: Standardize (standard scaling) numeric columns. 
- **DataAugmenterLSTM**: Augment data with Gaussian noise and reshape data into the format 3D **(samples, timesteps, features)** for LSTM.

In [ ]:
class DataTypeConverter(BaseEstimator, TransformerMixin):
    def __init__(self, numeric_cols, binary_cols):
        self.numeric_cols = numeric_cols
        self.binary_cols = binary_cols
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        for col in self.numeric_cols:
            X_copy[col] = pd.to_numeric(X_copy[col], errors='coerce')
        for col in self.binary_cols:
            X_copy[col] = pd.to_numeric(X_copy[col], errors='coerce').astype('int')
        return X_copy

# Transformer tùy chỉnh để xử lý NaN
class NaNImputer(BaseEstimator, TransformerMixin):
    def __init__(self, numeric_cols):
        self.numeric_cols = numeric_cols
        self.means_ = {}
    
    def fit(self, X, y=None):
        for col in self.numeric_cols:
            self.means_[col] = X[col].mean()
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        for col in self.numeric_cols:
            X_copy[col].fillna(self.means_[col], inplace=True)
        return X_copy

# Transformer tùy chỉnh để tiêu chuẩn hóa (chỉ các cột số)
class SelectiveScaler(BaseEstimator, TransformerMixin):
    def __init__(self, numeric_cols):
        self.numeric_cols = numeric_cols
        self.scaler_ = StandardScaler()
    
    def fit(self, X, y=None):
        self.scaler_.fit(X[self.numeric_cols])
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy[self.numeric_cols] = self.scaler_.transform(X_copy[self.numeric_cols])
        return X_copy

# Transformer tùy chỉnh để tăng cường dữ liệu và reshape cho LSTM
class DataAugmenterLSTM(BaseEstimator, TransformerMixin):
    def __init__(self, noise_factor=0.01, n_copies=2, timesteps=1):
        self.noise_factor = noise_factor
        self.n_copies = n_copies
        self.timesteps = timesteps
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_augmented = X.copy()
        y_augmented = y.copy() if y is not None else None
        for _ in range(self.n_copies):
            noise = np.random.normal(loc=0, scale=self.noise_factor, size=X.shape)
            X_noisy = X + noise
            X_augmented = np.vstack((X_augmented, X_noisy))
            if y is not None:
                y_augmented = np.hstack((y_augmented, y))
        # Reshape cho LSTM: (samples, timesteps, features)
        X_augmented = X_augmented.reshape(X_augmented.shape[0], self.timesteps, X_augmented.shape[1])
        return X_augmented, y_augmented

## Read and check the data.

In [ ]:
df = pd.read_csv('desharnais1.1_processed_corrected.csv')
numeric_columns = ['TeamExp', 'ManagerExp', 'YearEnd', 'Length', 
                   'Transactions', 'Entities', 'Adjustment', 'PointsAjust',
                   'StartYear', 'ProjectDurationYears', 'Transactions_Entities',
                   'Effort_PointsAjust', 'Effort_per_PointsAjust', 'Transactions_per_Entities']
binary_columns = ['Language_b\'1\'', 'Language_b\'2\'', 'Language_b\'3\'', 'HighEffort', 'HighPointsAjust']
features = numeric_columns + binary_columns
target = 'Effort'

## Create and apply a preprocessing pipeline

In [ ]:
X = df[features]
y = df[target].values

pipeline = Pipeline([
    ('data_type_converter', DataTypeConverter(numeric_cols=numeric_columns, binary_cols=binary_columns)),
    ('nan_imputer', NaNImputer(numeric_cols=numeric_columns)),
    ('scaler', SelectiveScaler(numeric_cols=numeric_columns)),
])

X_transformed = pipeline.fit_transform(X)

C:\Users\lequa\AppData\Local\Temp\ipykernel_11776\2711339975.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_copy[col].fillna(self.means_[col], inplace=True)


In [ ]:
data_augmenter = DataAugmenterLSTM(noise_factor=0.01, n_copies=2, timesteps=1)
X_augmented, y_augmented = data_augmenter.transform(X_transformed.values, y)

print("\n=== After data augmentation with Gaussian noise ===")
print("X_augmented shape:", X_augmented.shape)
print("y_augmented shape:", y_augmented.shape)


=== After data augmentation with Gaussian noise ===
X_augmented shape: (243, 1, 19)
y_augmented shape: (243,)


## Train/Test Split

Split the augmented data (X_augmented, y_augmented) into a training set (85%) and a test set (15%) using train_test_split with random_state=42.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_augmented, test_size=0.15, random_state=42)

print(f"\n✅ Data size of LSTM:")
print(f" - X_train: {X_train.shape}")
print(f" - X_test : {X_test.shape}")


✅ Data size of LSTM:
 - X_train: (206, 1, 19)
 - X_test : (37, 1, 19)


## Building LSTM model

Define the function build_lstm_model to create an LSTM model with the architecture: 
- **Input** layer with shape (timesteps, features). 
- **LSTM** layer with the number of units (lstm_units) and does not return sequences (return_sequences=False). 
- D**ropou**t layer to reduce overfitting. 
- **Dense** layer with dense_units and relu activation. 
- Second **Dropout** layer. 
- Final **Dense** layer with 1 unit and linear activation to predict Effort.

Use the **Huber** loss function and optimize with **Adam**.

In [ ]:
def build_lstm_model(lstm_units=32, dense_units=16, dropout_rate=0.3, learning_rate=0.001):
    model = Sequential([
        Input(shape=(X_train.shape[1], X_train.shape[2])),
        LSTM(lstm_units, return_sequences=False),
        Dropout(dropout_rate),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='linear')
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])
    return model

## Definition of hyperparameter space and PSO function

Define the hyperparameter space (param_bounds) for lstm_units, dense_units, dropout_rate, learning_rate, batch_size, epochs. 

Use two functions: 
- **random_particle**: Encode hyperparameters into a random vector (particle). 
- **decode_particle**: Decode the vector into a hyperparameter dictionary.

The fitness_function uses KFold cross-validation (3 folds) to assess the performance of each hyperparameter set based on RMSE.

In [ ]:
param_bounds = {
    'lstm_units': (16, 64),
    'dense_units': (8, 32),
    'dropout_rate': (0.2, 0.4),
    'learning_rate': (1e-4, 1e-2),
    'batch_size': (16, 32),
    'epochs': (50, 100)
}

def random_particle():
    return np.array([
        np.random.randint(param_bounds['lstm_units'][0], param_bounds['lstm_units'][1] + 1),
        np.random.randint(param_bounds['dense_units'][0], param_bounds['dense_units'][1] + 1),
        np.random.uniform(param_bounds['dropout_rate'][0], param_bounds['dropout_rate'][1]),
        np.random.uniform(param_bounds['learning_rate'][0], param_bounds['learning_rate'][1]),
        np.random.randint(param_bounds['batch_size'][0], param_bounds['batch_size'][1] + 1),
        np.random.randint(param_bounds['epochs'][0], param_bounds['epochs'][1] + 1)
    ])

def decode_particle(particle):
    params = {
        'lstm_units': int(particle[0]),
        'dense_units': int(particle[1]),
        'dropout_rate': particle[2],
        'learning_rate': particle[3],
        'batch_size': int(particle[4]),
        'epochs': int(particle[5])
    }
    params['dropout_rate'] = np.clip(params['dropout_rate'], param_bounds['dropout_rate'][0], param_bounds['dropout_rate'][1])
    return params

def fitness_function(particle):
    params = decode_particle(particle)
    model = build_lstm_model(**{k: v for k, v in params.items() if k != 'batch_size' and k != 'epochs'})
    
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    rmse_scores = []
    
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
        
        model.fit(X_tr, y_tr, epochs=params['epochs'], batch_size=params['batch_size'], 
                validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
        y_pred = model.predict(X_val, verbose=0)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_scores.append(rmse)
    
    return np.mean(rmse_scores)

## Implementing the PSO algorithm

Using the PSO algorithm to find the optimal hyperparameter set. 
- Initialize num_particles=15 and max_iter=10. 

- Update the position and velocity of the particles based on the PSO formula. 

- Find the best particle (g_best_position) and the best score (g_best_score).

In [ ]:
def run_pso_lstm(num_particles=15, max_iter=10):
    dim = len(param_bounds)
    bounds_array = np.array(list(param_bounds.values()))
    
    particles = [random_particle() for _ in range(num_particles)]
    velocities = [np.zeros(dim) for _ in range(num_particles)]
    
    p_best_positions = particles.copy()
    p_best_scores = [fitness_function(p) for p in particles]
    
    g_best_index = np.argmin(p_best_scores)
    g_best_position = p_best_positions[g_best_index]
    g_best_score = p_best_scores[g_best_index]
    
    w, c1, c2 = 0.5, 1.5, 1.5
    
    for iter in range(max_iter):
        print(f"\n🔁 Iteration {iter + 1}/{max_iter}")
        for i in range(num_particles):
            r1 = np.random.rand(dim)
            r2 = np.random.rand(dim)
            
            velocities[i] = (
                w * velocities[i]
                + c1 * r1 * (p_best_positions[i] - particles[i])
                + c2 * r2 * (g_best_position - particles[i])
            )
            
            particles[i] += velocities[i]
            particles[i] = np.clip(particles[i], bounds_array[:, 0], bounds_array[:, 1])
            particles[i][2] = np.clip(particles[i][2], param_bounds['dropout_rate'][0], param_bounds['dropout_rate'][1])
            
            score = fitness_function(particles[i])
            
            if score < p_best_scores[i]:
                p_best_scores[i] = score
                p_best_positions[i] = particles[i]
                
            if score < g_best_score:
                g_best_score = score
                g_best_position = particles[i]
                print(f"✅ Cập nhật g_best: Score = {g_best_score:.4f}")
    
    return g_best_position, g_best_score

# Chạy PSO
print("🚀 Chạy PSO để tìm siêu tham số tối ưu...")
best_particle, best_score = run_pso_lstm(num_particles=15, max_iter=10)
best_params = decode_particle(best_particle)
print(f"🏆 Siêu tham số tốt nhất: {best_params}")
print(f"📉 Score tốt nhất: {best_score:.4f}")

🚀 Chạy PSO để tìm siêu tham số tối ưu...

🔁 Iteration 1/10

🔁 Iteration 2/10

🔁 Iteration 3/10

🔁 Iteration 4/10

🔁 Iteration 5/10

🔁 Iteration 6/10
✅ Cập nhật g_best: Score = 0.0663

🔁 Iteration 7/10

🔁 Iteration 8/10

🔁 Iteration 9/10

🔁 Iteration 10/10
🏆 Siêu tham số tốt nhất: {'lstm_units': 57, 'dense_units': 20, 'dropout_rate': np.float64(0.2), 'learning_rate': np.float64(0.008488723679811452), 'batch_size': 28, 'epochs': 55}
📉 Score tốt nhất: 0.0663


## Optimal model training

- Use the best hyperparameters from PSO to train the LSTM model on the entire training set with KFold cross-validation (3 folds). 
- Use EarlyStopping and ReduceLROnPlateau to avoid overfitting and adjust the learning rate. 
- Save the training history (loss, val_loss) for analysis.

In [ ]:
model_optimal = build_lstm_model(**{k: v for k, v in best_params.items() if k != 'batch_size' and k != 'epochs'})
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

kf = KFold(n_splits=3, shuffle=True, random_state=42)
rmse_scores_optimal = []
history_all = {'loss': [], 'val_loss': []}

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"\n📂 Fold {fold + 1}/3")
    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]
    
    history = model_optimal.fit(X_tr, y_tr, epochs=best_params['epochs'], batch_size=best_params['batch_size'], 
                            validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
    y_pred = model_optimal.predict(X_val, verbose=0)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores_optimal.append(rmse)
    print(f"✅ Fold {fold + 1} RMSE: {rmse:.4f}")
    
    # Lưu lịch sử huấn luyện
    history_all['loss'].append(history.history['loss'])
    history_all['val_loss'].append(history.history['val_loss'])

print(f"\n📊 RMSE trung bình qua 3 folds: {np.mean(rmse_scores_optimal):.4f}")


📂 Fold 1/3
✅ Fold 1 RMSE: 0.0763

📂 Fold 2/3
✅ Fold 2 RMSE: 0.0844

📂 Fold 3/3
✅ Fold 3 RMSE: 0.0578

📊 RMSE trung bình qua 3 folds: 0.0728


## Model evaluation and bootstrap

In [ ]:
max_len = max(len(h) for h in history_all['loss'])
loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['loss']], axis=0)
val_loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['val_loss']], axis=0)

# Đánh giá trên tập test
y_pred = model_optimal.predict(X_test, verbose=0).flatten()

# Tính các chỉ số đánh giá
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = calculate_mape(y_test, y_pred)
mmre = calculate_mmre(y_test, y_pred)
mdmre = calculate_mdmre(y_test, y_pred)
pred25 = calculate_pred25(y_test, y_pred)

# Đánh giá bootstrap
n_bootstraps = 500
bootstrap_metrics = {'mse': [], 'mae': [], 'r2': [], 'mape': [], 'mmre': [], 'mdmre': [], 'pred25': []}

for _ in range(n_bootstraps):
    indices = np.random.choice(len(y_test), len(y_test), replace=True)
    y_test_boot = y_test[indices]
    y_pred_boot = y_pred[indices]
    bootstrap_metrics['mse'].append(mean_squared_error(y_test_boot, y_pred_boot))
    bootstrap_metrics['mae'].append(mean_absolute_error(y_test_boot, y_pred_boot))
    bootstrap_metrics['r2'].append(r2_score(y_test_boot, y_pred_boot))
    bootstrap_metrics['mape'].append(calculate_mape(y_test_boot, y_pred_boot))
    bootstrap_metrics['mmre'].append(calculate_mmre(y_test_boot, y_pred_boot))
    bootstrap_metrics['mdmre'].append(calculate_mdmre(y_test_boot, y_pred_boot))
    bootstrap_metrics['pred25'].append(calculate_pred25(y_test_boot, y_pred_boot))

# In kết quả
print("\n📈 Kết quả đánh giá bootstrap (trên giá trị đã scale):")
print(f"📌 MSE     : {np.mean(bootstrap_metrics['mse']):.4f} ± {np.std(bootstrap_metrics['mse']):.4f}")
print(f"📌 RMSE    : {np.mean(np.sqrt(bootstrap_metrics['mse'])):.4f} ± {np.std(np.sqrt(bootstrap_metrics['mse'])):.4f}")
print(f"📌 MAE     : {np.mean(bootstrap_metrics['mae']):.4f} ± {np.std(bootstrap_metrics['mae']):.4f}")
print(f"📌 R²      : {np.mean(bootstrap_metrics['r2']):.4f} ± {np.std(bootstrap_metrics['r2']):.4f}")
print(f"📌 MAPE    : {np.nanmean(bootstrap_metrics['mape']):.2f}% ± {np.nanstd(bootstrap_metrics['mape']):.2f}%")
print(f"📌 MMRE    : {np.nanmean(bootstrap_metrics['mmre']):.4f} ± {np.nanstd(bootstrap_metrics['mmre']):.4f}")
print(f"📌 MdMRE   : {np.nanmean(bootstrap_metrics['mdmre']):.4f} ± {np.nanstd(bootstrap_metrics['mdmre']):.4f}")
print(f"📌 PRED(25): {np.nanmean(bootstrap_metrics['pred25']):.2f}% ± {np.nanstd(bootstrap_metrics['pred25']):.2f}%")

# Lưu kết quả đánh giá
results = {
    'MSE': mse,
    'RMSE': rmse,
    'MAE': mae,
    'R2': r2,
    'MAPE': mape,
    'MMRE': mmre,
    'MdMRE': mdmre,
    'PRED(25)': pred25,
    'Bootstrap_MSE_Mean': np.mean(bootstrap_metrics['mse']),
    'Bootstrap_MSE_Std': np.std(bootstrap_metrics['mse']),
    'Bootstrap_MAE_Mean': np.mean(bootstrap_metrics['mae']),
    'Bootstrap_MAE_Std': np.std(bootstrap_metrics['mae']),
    'Bootstrap_R2_Mean': np.mean(bootstrap_metrics['r2']),
    'Bootstrap_R2_Std': np.std(bootstrap_metrics['r2']),
    'Bootstrap_MAPE_Mean': np.nanmean(bootstrap_metrics['mape']),
    'Bootstrap_MAPE_Std': np.nanstd(bootstrap_metrics['mape']),
    'Bootstrap_MMRE_Mean': np.nanmean(bootstrap_metrics['mmre']),
    'Bootstrap_MMRE_Std': np.nanstd(bootstrap_metrics['mmre']),
    'Bootstrap_MdMRE_Mean': np.nanmean(bootstrap_metrics['mdmre']),
    'Bootstrap_MdMRE_Std': np.nanstd(bootstrap_metrics['mdmre']),
    'Bootstrap_PRED25_Mean': np.nanmean(bootstrap_metrics['pred25']),
    'Bootstrap_PRED25_Std': np.nanstd(bootstrap_metrics['pred25'])
}

results_df = pd.DataFrame([results])
results_df.to_csv('lstm_evaluation_results_scaled.csv', index=False)
print("\nĐã lưu kết quả đánh giá vào 'lstm_evaluation_results_scaled.csv'")

# Trực quan hóa kết quả
plt.figure(figsize=(15, 12))

# Loss trung bình qua các folds
plt.subplot(2, 2, 1)
plt.plot(loss_avg, label='Training Loss')
plt.plot(val_loss_avg, label='Validation Loss')
plt.title('Average Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Huber Loss')
plt.legend()

# Predicted vs Actual
plt.subplot(2, 2, 2)
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.title('Predicted vs Actual Effort (Scaled)')
plt.xlabel('Actual Effort (Scaled)')
plt.ylabel('Predicted Effort (Scaled)')

# Error Distribution
errors = y_test - y_pred
plt.subplot(2, 2, 3)
sns.histplot(errors, kde=True)
plt.title('Error Distribution')
plt.xlabel('Prediction Error (Scaled)')
plt.ylabel('Frequency')

# Bootstrap RMSE
plt.subplot(2, 2, 4)
sns.boxplot(y=np.sqrt(bootstrap_metrics['mse']))
plt.title('Bootstrap RMSE Distribution (Scaled)')
plt.ylabel('RMSE (Scaled)')

plt.tight_layout()
plt.savefig('lstm_visualization_results_scaled.png')
plt.close()
print("\nĐã lưu hình ảnh trực quan hóa vào 'lstm_visualization_results_scaled.png'")


📈 Kết quả đánh giá bootstrap (trên giá trị đã scale):
📌 MSE     : 0.0086 ± 0.0029
📌 RMSE    : 0.0916 ± 0.0156
📌 MAE     : 0.0653 ± 0.0107
📌 R²      : 0.9937 ± 0.0021
📌 MAPE    : 32.12% ± 23.39%
📌 MMRE    : 0.3212 ± 0.2339
📌 MdMRE   : 0.0394 ± 0.0165
📌 PRED(25): 88.12% ± 7.85%

Đã lưu kết quả đánh giá vào 'lstm_evaluation_results_scaled.csv'

Đã lưu hình ảnh trực quan hóa vào 'lstm_visualization_results_scaled.png'


# MLP

## Import libraries and set the seed.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Thiết lập seed để tái lập
np.random.seed(42)
tf.random.set_seed(42)

## Define the evaluation functions.

In [ ]:
def calculate_mape(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def calculate_mmre(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    return np.mean(np.abs(y_true[mask] - y_pred[mask]) / y_true[mask])

def calculate_mdmre(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    return np.median(np.abs(y_true[mask] - y_pred[mask]) / y_true[mask])

def calculate_pred25(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    mre = np.abs(y_true[mask] - y_pred[mask]) / y_true[mask]
    return np.mean(mre <= 0.25) * 100

## Define custom Transformers.

MLP only enhances the data with Gaussian noise without reshaping. This means that the data stays in 2D form (samples, features)

In [ ]:
class NaNImputer(BaseEstimator, TransformerMixin):
    def __init__(self, numeric_cols):
        self.numeric_cols = numeric_cols
        self.means_ = {}
    
    def fit(self, X, y=None):
        for col in self.numeric_cols:
            self.means_[col] = X[col].mean()
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        for col in self.numeric_cols:
            X_copy[col].fillna(self.means_[col], inplace=True)
        return X_copy

# Transformer tùy chỉnh để tiêu chuẩn hóa (chỉ các cột số)
class SelectiveScaler(BaseEstimator, TransformerMixin):
    def __init__(self, numeric_cols):
        self.numeric_cols = numeric_cols
        self.scaler_ = StandardScaler()
    
    def fit(self, X, y=None):
        self.scaler_.fit(X[self.numeric_cols])
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy[self.numeric_cols] = self.scaler_.transform(X_copy[self.numeric_cols])
        return X_copy

# Transformer tùy chỉnh để tăng cường dữ liệu
class DataAugmenter(BaseEstimator, TransformerMixin):
    def __init__(self, noise_factor=0.01, n_copies=2):
        self.noise_factor = noise_factor
        self.n_copies = n_copies
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_augmented = X.copy()
        y_augmented = y.copy() if y is not None else None
        for _ in range(self.n_copies):
            noise = np.random.normal(loc=0, scale=self.noise_factor, size=X.shape)
            X_noisy = X + noise
            X_augmented = np.vstack((X_augmented, X_noisy))
            if y is not None:
                y_augmented = np.hstack((y_augmented, y))
        return X_augmented, y_augmented

## Read and check the data.

In [ ]:
df = pd.read_csv('desharnais1.1_processed_corrected.csv')

numeric_columns = ['TeamExp', 'ManagerExp', 'YearEnd', 'Length', 
                   'Transactions', 'Entities', 'Adjustment', 'PointsAjust',
                   'StartYear', 'ProjectDurationYears', 'Transactions_Entities',
                   'Effort_PointsAjust', 'Effort_per_PointsAjust', 'Transactions_per_Entities']
binary_columns = ['Language_b\'1\'', 'Language_b\'2\'', 'Language_b\'3\'', 'HighEffort', 'HighPointsAjust']
features = numeric_columns + binary_columns
target = 'Effort'

## Create and apply a preprocessing pipeline

In [ ]:
pipeline = Pipeline([
    ('data_type_converter', DataTypeConverter(numeric_cols=numeric_columns, binary_cols=binary_columns)),
    ('nan_imputer', NaNImputer(numeric_cols=numeric_columns)),
    ('scaler', SelectiveScaler(numeric_cols=numeric_columns)),
])

X = df[features]
y = df[target].values

X_transformed = pipeline.fit_transform(X)

data_augmenter = DataAugmenter(noise_factor=0.01, n_copies=2)
X_augmented, y_augmented = data_augmenter.transform(X_transformed.values, y)

print("\n=== After data augmentation with Gaussian noise ===")
print("X_augmented shape:", X_augmented.shape)
print("y_augmented shape:", y_augmented.shape)


=== After data augmentation with Gaussian noise ===
X_augmented shape: (243, 19)
y_augmented shape: (243,)


C:\Users\lequa\AppData\Local\Temp\ipykernel_11776\1988351987.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_copy[col].fillna(self.means_[col], inplace=True)


## Train/Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_augmented, test_size=0.15, random_state=42)

print(f"\n✅ Data size of MLP:")
print(f" - X_train: {X_train.shape}")
print(f" - X_test : {X_test.shape}")


✅ Data size of MLP:
 - X_train: (206, 19)
 - X_test : (37, 19)


## Building MLP model

Define the build_mlp_model function to create an MLP model with the following architecture:

- Input layer with shape (features,), i.e. 2D data.
- First Dense layer with dense_units_1 unit and relu activation.
- Dropout layer to reduce overfitting.
- Second Dense layer with dense_units_2 units and relu activation.
- Second Dropout layer.
-  Dense layer with 1 unit and linear activation to predict Effort.

Use Huber loss function and optimize with Adam.

In [ ]:
def build_mlp_model(dense_units_1=64, dense_units_2=32, dropout_rate=0.3, learning_rate=0.001):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(dense_units_1, activation='relu'),
        Dropout(dropout_rate),
        Dense(dense_units_2, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='linear')
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])
    return model

## Definition of hyperparameter space and PSO function

Define hyperparameter space (param_bounds) for dense_units_1, dense_units_2, dropout_rate, learning_rate, batch_size, epochs.

In [ ]:
param_bounds = {
    'dense_units_1': (32, 128),
    'dense_units_2': (16, 64),
    'dropout_rate': (0.2, 0.4),
    'learning_rate': (1e-4, 1e-2),
    'batch_size': (16, 32),
    'epochs': (50, 100)
}

def random_particle():
    return np.array([
        np.random.randint(param_bounds['dense_units_1'][0], param_bounds['dense_units_1'][1] + 1),
        np.random.randint(param_bounds['dense_units_2'][0], param_bounds['dense_units_2'][1] + 1),
        np.random.uniform(param_bounds['dropout_rate'][0], param_bounds['dropout_rate'][1]),
        np.random.uniform(param_bounds['learning_rate'][0], param_bounds['learning_rate'][1]),
        np.random.randint(param_bounds['batch_size'][0], param_bounds['batch_size'][1] + 1),
        np.random.randint(param_bounds['epochs'][0], param_bounds['epochs'][1] + 1)
    ])

def decode_particle(particle):
    params = {
        'dense_units_1': int(particle[0]),
        'dense_units_2': int(particle[1]),
        'dropout_rate': particle[2],
        'learning_rate': particle[3],
        'batch_size': int(particle[4]),
        'epochs': int(particle[5])
    }
    params['dropout_rate'] = np.clip(params['dropout_rate'], param_bounds['dropout_rate'][0], param_bounds['dropout_rate'][1])
    return params

# Hàm fitness cho PSO
def fitness_function(particle):
    params = decode_particle(particle)
    model = build_mlp_model(**{k: v for k, v in params.items() if k != 'batch_size' and k != 'epochs'})
    
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    rmse_scores = []
    
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
        
        model.fit(X_tr, y_tr, epochs=params['epochs'], batch_size=params['batch_size'], 
                validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
        y_pred = model.predict(X_val, verbose=0)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_scores.append(rmse)
    
    return np.mean(rmse_scores)

## Implementing the PSO algorithm

In [ ]:
def run_pso_mlp(num_particles=15, max_iter=10):
    dim = len(param_bounds)
    bounds_array = np.array(list(param_bounds.values()))
    
    particles = [random_particle() for _ in range(num_particles)]
    velocities = [np.zeros(dim) for _ in range(num_particles)]
    
    p_best_positions = particles.copy()
    p_best_scores = [fitness_function(p) for p in particles]
    
    g_best_index = np.argmin(p_best_scores)
    g_best_position = p_best_positions[g_best_index]
    g_best_score = p_best_scores[g_best_index]
    
    w, c1, c2 = 0.5, 1.5, 1.5
    
    for iter in range(max_iter):
        print(f"\n🔁 Iteration {iter + 1}/{max_iter}")
        for i in range(num_particles):
            r1 = np.random.rand(dim)
            r2 = np.random.rand(dim)
            
            velocities[i] = (
                w * velocities[i]
                + c1 * r1 * (p_best_positions[i] - particles[i])
                + c2 * r2 * (g_best_position - particles[i])
            )
            
            particles[i] += velocities[i]
            particles[i] = np.clip(particles[i], bounds_array[:, 0], bounds_array[:, 1])
            particles[i][2] = np.clip(particles[i][2], param_bounds['dropout_rate'][0], param_bounds['dropout_rate'][1])
            
            score = fitness_function(particles[i])
            
            if score < p_best_scores[i]:
                p_best_scores[i] = score
                p_best_positions[i] = particles[i]
                
            if score < g_best_score:
                g_best_score = score
                g_best_position = particles[i]
                print(f"✅ Cập nhật g_best: Score = {g_best_score:.4f}")
    
    return g_best_position, g_best_score

# Chạy PSO
print("🚀 Chạy PSO để tìm siêu tham số tối ưu...")
best_particle, best_score = run_pso_mlp(num_particles=15, max_iter=10)
best_params = decode_particle(best_particle)
print(f"🏆 Siêu tham số tốt nhất: {best_params}")
print(f"📉 Score tốt nhất: {best_score:.4f}")

🚀 Chạy PSO để tìm siêu tham số tối ưu...

🔁 Iteration 1/10
✅ Cập nhật g_best: Score = 0.1050
✅ Cập nhật g_best: Score = 0.0999
✅ Cập nhật g_best: Score = 0.0952
✅ Cập nhật g_best: Score = 0.0921

🔁 Iteration 2/10
✅ Cập nhật g_best: Score = 0.0832

🔁 Iteration 3/10
✅ Cập nhật g_best: Score = 0.0772

🔁 Iteration 4/10
✅ Cập nhật g_best: Score = 0.0765

🔁 Iteration 5/10

🔁 Iteration 6/10

🔁 Iteration 7/10

🔁 Iteration 8/10
✅ Cập nhật g_best: Score = 0.0761

🔁 Iteration 9/10
✅ Cập nhật g_best: Score = 0.0724
✅ Cập nhật g_best: Score = 0.0704

🔁 Iteration 10/10
🏆 Siêu tham số tốt nhất: {'dense_units_1': 132, 'dense_units_2': 71, 'dropout_rate': np.float64(0.2), 'learning_rate': np.float64(0.0072998326507956734), 'batch_size': 28, 'epochs': 100}
📉 Score tốt nhất: 0.0704


## Optimal model training

In [ ]:
model_optimal = build_mlp_model(**{k: v for k, v in best_params.items() if k != 'batch_size' and k != 'epochs'})
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

kf = KFold(n_splits=3, shuffle=True, random_state=42)
rmse_scores_optimal = []
history_all = {'loss': [], 'val_loss': []}

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"\n📂 Fold {fold + 1}/3")
    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]
    
    history = model_optimal.fit(X_tr, y_tr, epochs=best_params['epochs'], batch_size=best_params['batch_size'], 
                            validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
    y_pred = model_optimal.predict(X_val, verbose=0)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores_optimal.append(rmse)
    print(f"✅ Fold {fold + 1} RMSE: {rmse:.4f}")
    
    # Lưu lịch sử huấn luyện
    history_all['loss'].append(history.history['loss'])
    history_all['val_loss'].append(history.history['val_loss'])

print(f"\n📊 RMSE trung bình qua 3 folds: {np.mean(rmse_scores_optimal):.4f}")


📂 Fold 1/3
✅ Fold 1 RMSE: 0.0870

📂 Fold 2/3
✅ Fold 2 RMSE: 0.0782

📂 Fold 3/3
✅ Fold 3 RMSE: 0.0647

📊 RMSE trung bình qua 3 folds: 0.0766


## Model evaluation and bootstrap

In [ ]:
max_len = max(len(h) for h in history_all['loss'])
loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['loss']], axis=0)
val_loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['val_loss']], axis=0)

# Đánh giá trên tập test
y_pred = model_optimal.predict(X_test, verbose=0).flatten()

# Tính các chỉ số đánh giá
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = calculate_mape(y_test, y_pred)
mmre = calculate_mmre(y_test, y_pred)
mdmre = calculate_mdmre(y_test, y_pred)
pred25 = calculate_pred25(y_test, y_pred)

# Đánh giá bootstrap
n_bootstraps = 500
bootstrap_metrics = {'mse': [], 'mae': [], 'r2': [], 'mape': [], 'mmre': [], 'mdmre': [], 'pred25': []}

for _ in range(n_bootstraps):
    indices = np.random.choice(len(y_test), len(y_test), replace=True)
    y_test_boot = y_test[indices]
    y_pred_boot = y_pred[indices]
    bootstrap_metrics['mse'].append(mean_squared_error(y_test_boot, y_pred_boot))
    bootstrap_metrics['mae'].append(mean_absolute_error(y_test_boot, y_pred_boot))
    bootstrap_metrics['r2'].append(r2_score(y_test_boot, y_pred_boot))
    bootstrap_metrics['mape'].append(calculate_mape(y_test_boot, y_pred_boot))
    bootstrap_metrics['mmre'].append(calculate_mmre(y_test_boot, y_pred_boot))
    bootstrap_metrics['mdmre'].append(calculate_mdmre(y_test_boot, y_pred_boot))
    bootstrap_metrics['pred25'].append(calculate_pred25(y_test_boot, y_pred_boot))

# In kết quả
print("\n📈 Kết quả đánh giá bootstrap (trên giá trị đã scale):")
print(f"📌 MSE     : {np.mean(bootstrap_metrics['mse']):.4f} ± {np.std(bootstrap_metrics['mse']):.4f}")
print(f"📌 RMSE    : {np.mean(np.sqrt(bootstrap_metrics['mse'])):.4f} ± {np.std(np.sqrt(bootstrap_metrics['mse'])):.4f}")
print(f"📌 MAE     : {np.mean(bootstrap_metrics['mae']):.4f} ± {np.std(bootstrap_metrics['mae']):.4f}")
print(f"📌 R²      : {np.mean(bootstrap_metrics['r2']):.4f} ± {np.std(bootstrap_metrics['r2']):.4f}")
print(f"📌 MAPE    : {np.mean(bootstrap_metrics['mape']):.2f}% ± {np.std(bootstrap_metrics['mape']):.2f}%")
print(f"📌 MMRE    : {np.mean(bootstrap_metrics['mmre']):.4f} ± {np.std(bootstrap_metrics['mmre']):.4f}")
print(f"📌 MdMRE   : {np.mean(bootstrap_metrics['mdmre']):.4f} ± {np.std(bootstrap_metrics['mdmre']):.4f}")
print(f"📌 PRED(25): {np.mean(bootstrap_metrics['pred25']):.2f}% ± {np.std(bootstrap_metrics['pred25']):.2f}%")

# Lưu kết quả đánh giá
results = {
    'MSE': mse,
    'RMSE': rmse,
    'MAE': mae,
    'R2': r2,
    'MAPE': mape,
    'MMRE': mmre,
    'MdMRE': mdmre,
    'PRED(25)': pred25,
    'Bootstrap_MSE_Mean': np.mean(bootstrap_metrics['mse']),
    'Bootstrap_MSE_Std': np.std(bootstrap_metrics['mse']),
    'Bootstrap_MAE_Mean': np.mean(bootstrap_metrics['mae']),
    'Bootstrap_MAE_Std': np.std(bootstrap_metrics['mae']),
    'Bootstrap_R2_Mean': np.mean(bootstrap_metrics['r2']),
    'Bootstrap_R2_Std': np.std(bootstrap_metrics['r2']),
    'Bootstrap_MAPE_Mean': np.mean(bootstrap_metrics['mape']),
    'Bootstrap_MAPE_Std': np.std(bootstrap_metrics['mape']),
    'Bootstrap_MMRE_Mean': np.mean(bootstrap_metrics['mmre']),
    'Bootstrap_MMRE_Std': np.std(bootstrap_metrics['mmre']),
    'Bootstrap_MdMRE_Mean': np.mean(bootstrap_metrics['mdmre']),
    'Bootstrap_MdMRE_Std': np.std(bootstrap_metrics['mdmre']),
    'Bootstrap_PRED25_Mean': np.mean(bootstrap_metrics['pred25']),
    'Bootstrap_PRED25_Std': np.std(bootstrap_metrics['pred25'])
}

results_df = pd.DataFrame([results])
results_df.to_csv('mlp_evaluation_results_scaled.csv', index=False)
print("\nĐã lưu kết quả đánh giá vào 'mlp_evaluation_results_scaled.csv'")

# Trực quan hóa kết quả
plt.figure(figsize=(15, 12))

# Loss trung bình qua các folds
plt.subplot(2, 2, 1)
plt.plot(loss_avg, label='Training Loss')
plt.plot(val_loss_avg, label='Validation Loss')
plt.title('Average Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Huber Loss')
plt.legend()

# Predicted vs Actual
plt.subplot(2, 2, 2)
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.title('Predicted vs Actual Effort (Scaled)')
plt.xlabel('Actual Effort (Scaled)')
plt.ylabel('Predicted Effort (Scaled)')

# Error Distribution
errors = y_test - y_pred
plt.subplot(2, 2, 3)
sns.histplot(errors, kde=True)
plt.title('Error Distribution')
plt.xlabel('Prediction Error (Scaled)')
plt.ylabel('Frequency')

# Bootstrap RMSE
plt.subplot(2, 2, 4)
sns.boxplot(y=np.sqrt(bootstrap_metrics['mse']))
plt.title('Bootstrap RMSE Distribution (Scaled)')
plt.ylabel('RMSE (Scaled)')

plt.tight_layout()
plt.savefig('mlp_visualization_results_scaled.png')
plt.close()
print("\nĐã lưu hình ảnh trực quan hóa vào 'mlp_visualization_results_scaled.png'")


📈 Kết quả đánh giá bootstrap (trên giá trị đã scale):
📌 MSE     : 0.0116 ± 0.0041
📌 RMSE    : 0.1062 ± 0.0190
📌 MAE     : 0.0778 ± 0.0117
📌 R²      : 0.9915 ± 0.0027
📌 MAPE    : 82.88% ± 67.53%
📌 MMRE    : 0.8288 ± 0.6753
📌 MdMRE   : 0.0710 ± 0.0376
📌 PRED(25): 82.21% ± 9.36%

Đã lưu kết quả đánh giá vào 'mlp_evaluation_results_scaled.csv'

Đã lưu hình ảnh trực quan hóa vào 'mlp_visualization_results_scaled.png'


# RBFN

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Layer

# Thiết lập seed để tái lập
np.random.seed(42)
tf.random.set_seed(42)

## Define the evaluation functions

In [11]:
def calculate_mape(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def calculate_mmre(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    return np.mean(np.abs(y_true[mask] - y_pred[mask]) / y_true[mask])

def calculate_mdmre(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    return np.median(np.abs(y_true[mask] - y_pred[mask]) / y_true[mask])

def calculate_pred25(y_true, y_pred):
    mask = y_true > 0
    if np.sum(mask) == 0:
        return np.nan
    mre = np.abs(y_true[mask] - y_pred[mask]) / y_true[mask]
    return np.mean(mre <= 0.25) * 100

## Define custom Transformers

RBFN only enhances the data with Gaussian noise without reshaping. This means that the data is kept in 2D form (samples, features)

In [12]:
class DataTypeConverter(BaseEstimator, TransformerMixin):
    def __init__(self, numeric_cols, binary_cols):
        self.numeric_cols = numeric_cols
        self.binary_cols = binary_cols
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        for col in self.numeric_cols:
            X_copy[col] = pd.to_numeric(X_copy[col], errors='coerce')
        for col in self.binary_cols:
            X_copy[col] = pd.to_numeric(X_copy[col], errors='coerce').astype('int')
        return X_copy

# Transformer tùy chỉnh để xử lý NaN
class NaNImputer(BaseEstimator, TransformerMixin):
    def __init__(self, numeric_cols):
        self.numeric_cols = numeric_cols
        self.means_ = {}
    
    def fit(self, X, y=None):
        for col in self.numeric_cols:
            self.means_[col] = X[col].mean()
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        for col in self.numeric_cols:
            X_copy[col].fillna(self.means_[col], inplace=True)
        return X_copy

# Transformer tùy chỉnh để tiêu chuẩn hóa (chỉ các cột số)
class SelectiveScaler(BaseEstimator, TransformerMixin):
    def __init__(self, numeric_cols):
        self.numeric_cols = numeric_cols
        self.scaler_ = StandardScaler()
    
    def fit(self, X, y=None):
        self.scaler_.fit(X[self.numeric_cols])
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy[self.numeric_cols] = self.scaler_.transform(X_copy[self.numeric_cols])
        return X_copy

# Transformer tùy chỉnh để tăng cường dữ liệu
class DataAugmenter(BaseEstimator, TransformerMixin):
    def __init__(self, noise_factor=0.01, n_copies=2):
        self.noise_factor = noise_factor
        self.n_copies = n_copies
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_augmented = X.copy()
        y_augmented = y.copy() if y is not None else None
        for _ in range(self.n_copies):
            noise = np.random.normal(loc=0, scale=self.noise_factor, size=X.shape)
            X_noisy = X + noise
            X_augmented = np.vstack((X_augmented, X_noisy))
            if y is not None:
                y_augmented = np.hstack((y_augmented, y))
        return X_augmented, y_augmented

## Read data

In [13]:
df = pd.read_csv('desharnais1.1_processed_corrected.csv')

numeric_columns = ['TeamExp', 'ManagerExp', 'YearEnd', 'Length', 
                   'Transactions', 'Entities', 'Adjustment', 'PointsAjust',
                   'StartYear', 'ProjectDurationYears', 'Transactions_Entities',
                   'Effort_PointsAjust', 'Effort_per_PointsAjust', 'Transactions_per_Entities']
binary_columns = ['Language_b\'1\'', 'Language_b\'2\'', 'Language_b\'3\'', 'HighEffort', 'HighPointsAjust']
features = numeric_columns + binary_columns
target = 'Effort'

## Create and apply a preprocessing pipeline

In [14]:
pipeline = Pipeline([
    ('data_type_converter', DataTypeConverter(numeric_cols=numeric_columns, binary_cols=binary_columns)),
    ('nan_imputer', NaNImputer(numeric_cols=numeric_columns)),
    ('scaler', SelectiveScaler(numeric_cols=numeric_columns)),
])

X = df[features]
y = df[target].values

X_transformed = pipeline.fit_transform(X)

data_augmenter = DataAugmenter(noise_factor=0.01, n_copies=2)
X_augmented, y_augmented = data_augmenter.transform(X_transformed.values, y)

print("\n=== After data augmentation ===")
print("X_augmented shape:", X_augmented.shape)
print("y_augmented shape:", y_augmented.shape)


=== After data augmentation ===
X_augmented shape: (243, 19)
y_augmented shape: (243,)


C:\Users\lequa\AppData\Local\Temp\ipykernel_5832\3040840636.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_copy[col].fillna(self.means_[col], inplace=True)


## Train/Test Split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_augmented, test_size=0.15, random_state=42)

print(f"\n✅ Kích thước dữ liệu RBFN:")
print(f" - X_train: {X_train.shape}")
print(f" - X_test : {X_test.shape}")


✅ Kích thước dữ liệu RBFN:
 - X_train: (206, 19)
 - X_test : (37, 19)


## Building RBFN model


Define the build_rbfn_model function to create an RBFN model with the following architecture:

- Input layer with shape (features,), i.e. 2D data.
- Custom RBFLayer layer with n_centers centers and sigma parameter, implementing radial (Gaussian) basis function.
- Final Dense layer with 1 unit and linear activation to predict Effort.

Using Huber loss function and optimizing with Adam.


RBFN: Uses the RBFLayer class to implement radial (Gaussian) basis functions, based on the Euclidean distance between the input and the centers. This class is specific to RBFN and does not appear in other models.

In [16]:
class RBFLayer(Layer):
    def __init__(self, n_centers, sigma=1.0, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.n_centers = n_centers
        self.sigma = sigma
    
    def build(self, input_shape):
        self.centers = self.add_weight(name='centers',
                                      shape=(self.n_centers, input_shape[1]),
                                      initializer='glorot_uniform',
                                      trainable=True)
        self.sigma = self.add_weight(name='sigma',
                                    shape=(1,),
                                    initializer=tf.keras.initializers.Constant(self.sigma),
                                    trainable=True)
        super(RBFLayer, self).build(input_shape)
    
    def call(self, inputs):
        # Tính khoảng cách Euclidean bình phương
        diff = tf.expand_dims(inputs, axis=1) - tf.expand_dims(self.centers, axis=0)
        l2 = tf.reduce_sum(tf.square(diff), axis=-1)
        # Hàm Gaussian
        return tf.exp(-l2 / (2.0 * tf.square(self.sigma)))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.n_centers)

def build_rbfn_model(n_centers=20, sigma=1.0, learning_rate=0.001):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        RBFLayer(n_centers, sigma=sigma),
        Dense(1, activation='linear')
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])
    return model

## Definition of hyperparameter space and PSO function

Use n_centers (number of centers in RBFLayer) and sigma (parameter that adjusts the width of the Gaussian function), which are RBFN-specific hyperparameters.

In [17]:
param_bounds = {
    'n_centers': (10, 50),
    'sigma': (0.1, 2.0),
    'learning_rate': (1e-4, 1e-2),
    'batch_size': (16, 32),
    'epochs': (50, 100)
}

# Hàm mã hóa & giải mã particle
def random_particle():
    return np.array([
        np.random.randint(param_bounds['n_centers'][0], param_bounds['n_centers'][1] + 1),
        np.random.uniform(param_bounds['sigma'][0], param_bounds['sigma'][1]),
        np.random.uniform(param_bounds['learning_rate'][0], param_bounds['learning_rate'][1]),
        np.random.randint(param_bounds['batch_size'][0], param_bounds['batch_size'][1] + 1),
        np.random.randint(param_bounds['epochs'][0], param_bounds['epochs'][1] + 1)
    ])

def decode_particle(particle):
    params = {
        'n_centers': int(particle[0]),
        'sigma': particle[1],
        'learning_rate': particle[2],
        'batch_size': int(particle[3]),
        'epochs': int(particle[4])
    }
    params['sigma'] = np.clip(params['sigma'], param_bounds['sigma'][0], param_bounds['sigma'][1])
    return params

# Hàm fitness cho PSO
def fitness_function(particle):
    params = decode_particle(particle)
    model = build_rbfn_model(**{k: v for k, v in params.items() if k != 'batch_size' and k != 'epochs'})
    
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    rmse_scores = []
    
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
        
        model.fit(X_tr, y_tr, epochs=params['epochs'], batch_size=params['batch_size'], 
                validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
        y_pred = model.predict(X_val, verbose=0)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_scores.append(rmse)
    
    return np.mean(rmse_scores)

## Implementing the PSO algorithm

In [18]:
def run_pso_rbfn(num_particles=15, max_iter=10):
    dim = len(param_bounds)
    bounds_array = np.array(list(param_bounds.values()))
    
    particles = [random_particle() for _ in range(num_particles)]
    velocities = [np.zeros(dim) for _ in range(num_particles)]
    
    p_best_positions = particles.copy()
    p_best_scores = [fitness_function(p) for p in particles]
    
    g_best_index = np.argmin(p_best_scores)
    g_best_position = p_best_positions[g_best_index]
    g_best_score = p_best_scores[g_best_index]
    
    w, c1, c2 = 0.5, 1.5, 1.5
    
    for iter in range(max_iter):
        print(f"\n🔁 Iteration {iter + 1}/{max_iter}")
        for i in range(num_particles):
            r1 = np.random.rand(dim)
            r2 = np.random.rand(dim)
            
            velocities[i] = (
                w * velocities[i]
                + c1 * r1 * (p_best_positions[i] - particles[i])
                + c2 * r2 * (g_best_position - particles[i])
            )
            
            particles[i] += velocities[i]
            particles[i] = np.clip(particles[i], bounds_array[:, 0], bounds_array[:, 1])
            particles[i][1] = np.clip(particles[i][1], param_bounds['sigma'][0], param_bounds['sigma'][1])
            
            score = fitness_function(particles[i])
            
            if score < p_best_scores[i]:
                p_best_scores[i] = score
                p_best_positions[i] = particles[i]
                
            if score < g_best_score:
                g_best_score = score
                g_best_position = particles[i]
                print(f"✅ Cập nhật g_best: Score = {g_best_score:.4f}")
    
    return g_best_position, g_best_score

# Chạy PSO
print("🚀 Chạy PSO để tìm siêu tham số tối ưu...")
best_particle, best_score = run_pso_rbfn(num_particles=15, max_iter=10)
best_params = decode_particle(best_particle)
print(f"🏆 Siêu tham số tốt nhất: {best_params}")
print(f"📉 Score tốt nhất: {best_score:.4f}")

🚀 Chạy PSO để tìm siêu tham số tối ưu...

🔁 Iteration 1/10

🔁 Iteration 2/10
✅ Cập nhật g_best: Score = 0.1346
✅ Cập nhật g_best: Score = 0.1313


KeyboardInterrupt: 

## Optimal model training

In [ ]:
model_optimal = build_rbfn_model(**{k: v for k, v in best_params.items() if k != 'batch_size' and k != 'epochs'})
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

kf = KFold(n_splits=3, shuffle=True, random_state=42)
rmse_scores_optimal = []
history_all = {'loss': [], 'val_loss': []}

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"\n📂 Fold {fold + 1}/3")
    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]
    
    history = model_optimal.fit(X_tr, y_tr, epochs=best_params['epochs'], batch_size=best_params['batch_size'], 
                            validation_split=0.2, verbose=0, callbacks=[early_stopping, reduce_lr])
    y_pred = model_optimal.predict(X_val, verbose=0)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores_optimal.append(rmse)
    print(f"✅ Fold {fold + 1} RMSE: {rmse:.4f}")
    
    # Lưu lịch sử huấn luyện
    history_all['loss'].append(history.history['loss'])
    history_all['val_loss'].append(history.history['val_loss'])

print(f"\n📊 RMSE trung bình qua 3 folds: {np.mean(rmse_scores_optimal):.4f}")


📂 Fold 1/3
✅ Fold 1 RMSE: 0.1113

📂 Fold 2/3
✅ Fold 2 RMSE: 0.1488

📂 Fold 3/3
✅ Fold 3 RMSE: 0.0943

📊 RMSE trung bình qua 3 folds: 0.1181


## Model evaluation and bootstrap

In [ ]:
max_len = max(len(h) for h in history_all['loss'])
loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['loss']], axis=0)
val_loss_avg = np.mean([np.pad(h, (0, max_len - len(h)), 'constant', constant_values=np.nan) for h in history_all['val_loss']], axis=0)

# Đánh giá trên tập test
y_pred = model_optimal.predict(X_test, verbose=0).flatten()

# Tính các chỉ số đánh giá
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = calculate_mape(y_test, y_pred)
mmre = calculate_mmre(y_test, y_pred)
mdmre = calculate_mdmre(y_test, y_pred)
pred25 = calculate_pred25(y_test, y_pred)

# Đánh giá bootstrap
n_bootstraps = 500
bootstrap_metrics = {'mse': [], 'mae': [], 'r2': [], 'mape': [], 'mmre': [], 'mdmre': [], 'pred25': []}

for _ in range(n_bootstraps):
    indices = np.random.choice(len(y_test), len(y_test), replace=True)
    y_test_boot = y_test[indices]
    y_pred_boot = y_pred[indices]
    bootstrap_metrics['mse'].append(mean_squared_error(y_test_boot, y_pred_boot))
    bootstrap_metrics['mae'].append(mean_absolute_error(y_test_boot, y_pred_boot))
    bootstrap_metrics['r2'].append(r2_score(y_test_boot, y_pred_boot))
    bootstrap_metrics['mape'].append(calculate_mape(y_test_boot, y_pred_boot))
    bootstrap_metrics['mmre'].append(calculate_mmre(y_test_boot, y_pred_boot))
    bootstrap_metrics['mdmre'].append(calculate_mdmre(y_test_boot, y_pred_boot))
    bootstrap_metrics['pred25'].append(calculate_pred25(y_test_boot, y_pred_boot))

# In kết quả
print("\n📈 Kết quả đánh giá bootstrap (trên giá trị đã scale):")
print(f"📌 MSE     : {np.mean(bootstrap_metrics['mse']):.4f} ± {np.std(bootstrap_metrics['mse']):.4f}")
print(f"📌 RMSE    : {np.mean(np.sqrt(bootstrap_metrics['mse'])):.4f} ± {np.std(np.sqrt(bootstrap_metrics['mse'])):.4f}")
print(f"📌 MAE     : {np.mean(bootstrap_metrics['mae']):.4f} ± {np.std(bootstrap_metrics['mae']):.4f}")
print(f"📌 R²      : {np.mean(bootstrap_metrics['r2']):.4f} ± {np.std(bootstrap_metrics['r2']):.4f}")
print(f"📌 MAPE    : {np.nanmean(bootstrap_metrics['mape']):.2f}% ± {np.nanstd(bootstrap_metrics['mape']):.2f}%")
print(f"📌 MMRE    : {np.nanmean(bootstrap_metrics['mmre']):.4f} ± {np.nanstd(bootstrap_metrics['mmre']):.4f}")
print(f"📌 MdMRE   : {np.nanmean(bootstrap_metrics['mdmre']):.4f} ± {np.nanstd(bootstrap_metrics['mdmre']):.4f}")
print(f"📌 PRED(25): {np.nanmean(bootstrap_metrics['pred25']):.2f}% ± {np.nanstd(bootstrap_metrics['pred25']):.2f}%")

# Lưu kết quả đánh giá
results = {
    'MSE': mse,
    'RMSE': rmse,
    'MAE': mae,
    'R2': r2,
    'MAPE': mape,
    'MMRE': mmre,
    'MdMRE': mdmre,
    'PRED(25)': pred25,
    'Bootstrap_MSE_Mean': np.mean(bootstrap_metrics['mse']),
    'Bootstrap_MSE_Std': np.std(bootstrap_metrics['mse']),
    'Bootstrap_MAE_Mean': np.mean(bootstrap_metrics['mae']),
    'Bootstrap_MAE_Std': np.std(bootstrap_metrics['mae']),
    'Bootstrap_R2_Mean': np.mean(bootstrap_metrics['r2']),
    'Bootstrap_R2_Std': np.std(bootstrap_metrics['r2']),
    'Bootstrap_MAPE_Mean': np.nanmean(bootstrap_metrics['mape']),
    'Bootstrap_MAPE_Std': np.nanstd(bootstrap_metrics['mape']),
    'Bootstrap_MMRE_Mean': np.nanmean(bootstrap_metrics['mmre']),
    'Bootstrap_MMRE_Std': np.nanstd(bootstrap_metrics['mmre']),
    'Bootstrap_MdMRE_Mean': np.nanmean(bootstrap_metrics['mdmre']),
    'Bootstrap_MdMRE_Std': np.nanstd(bootstrap_metrics['mdmre']),
    'Bootstrap_PRED25_Mean': np.nanmean(bootstrap_metrics['pred25']),
    'Bootstrap_PRED25_Std': np.nanstd(bootstrap_metrics['pred25'])
}

results_df = pd.DataFrame([results])
results_df.to_csv('rbfn_evaluation_results_scaled.csv', index=False)
print("\nĐã lưu kết quả đánh giá vào 'rbfn_evaluation_results_scaled.csv'")

# Trực quan hóa kết quả
plt.figure(figsize=(15, 12))

# Loss trung bình qua các folds
plt.subplot(2, 2, 1)
plt.plot(loss_avg, label='Training Loss')
plt.plot(val_loss_avg, label='Validation Loss')
plt.title('Average Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Huber Loss')
plt.legend()

# Predicted vs Actual
plt.subplot(2, 2, 2)
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.title('Predicted vs Actual Effort (Scaled)')
plt.xlabel('Actual Effort (Scaled)')
plt.ylabel('Predicted Effort (Scaled)')

# Error Distribution
errors = y_test - y_pred
plt.subplot(2, 2, 3)
sns.histplot(errors, kde=True)
plt.title('Error Distribution')
plt.xlabel('Prediction Error (Scaled)')
plt.ylabel('Frequency')

# Bootstrap RMSE
plt.subplot(2, 2, 4)
sns.boxplot(y=np.sqrt(bootstrap_metrics['mse']))
plt.title('Bootstrap RMSE Distribution (Scaled)')
plt.ylabel('RMSE (Scaled)')

plt.tight_layout()
plt.savefig('rbfn_visualization_results_scaled.png')
plt.close()
print("\nĐã lưu hình ảnh trực quan hóa vào 'rbfn_visualization_results_scaled.png'")


📈 Kết quả đánh giá bootstrap (trên giá trị đã scale):
📌 MSE     : 0.0116 ± 0.0040
📌 RMSE    : 0.1060 ± 0.0187
📌 MAE     : 0.0777 ± 0.0118
📌 R²      : 0.9916 ± 0.0026
📌 MAPE    : 82.30% ± 67.88%
📌 MMRE    : 0.8230 ± 0.6788
📌 MdMRE   : 0.0715 ± 0.0363
📌 PRED(25): 82.35% ± 9.36%

Đã lưu kết quả đánh giá vào 'rbfn_evaluation_results_scaled.csv'

Đã lưu hình ảnh trực quan hóa vào 'rbfn_visualization_results_scaled.png'
